In [1]:
require 'hdf5';
require 'rnn';

In [2]:
myFile = hdf5.open('../data_preprocessed/6-grams.hdf5','r')
data = myFile:all()
myFile:close()

In [3]:
data['input_data_train']:size(1)-4

599905	


In [4]:
train = data['input_data_train']:narrow(1,5,data['input_data_train']:size(1)-909)
out = data['output_matrix_train']:narrow(1,1,data['input_data_train']:size(1)-909)

In [5]:
l = 50
b = 10
n = train:size(1)
train_matrix = train:view(b,n/b):type('torch.DoubleTensor')
out_matrix = out:view(b,n/b):type('torch.DoubleTensor')

In [6]:
nchar = 49
embed_dim = 16
LT = nn.LookupTable(nchar, embed_dim)

r = nn.Recurrent(
   embed_dim, nn.Linear(embed_dim, embed_dim), 
   nn.Linear(embed_dim, embed_dim), nn.Tanh(),l
)

rnn = nn.Sequential()
   :add(LT) 
   :add(nn.SplitTable(1,10))
   :add(nn.Sequencer(r))
   :add(nn.Sequencer(nn.Linear(embed_dim,2)))
   :add(nn.Sequencer(nn.LogSoftMax()))

rnn:remember('both')

crit = nn.SequencerCriterion(nn.ClassNLLCriterion())

params, grad_params = rnn:getParameters()

In [7]:
for i = 1, params:size(1) do
    params[i] = torch.uniform(-0.05,0.05)
end

In [ ]:
-- To store the loss
av_L = 0

for i = 1, 1 do
    -- timing the epoch
    timer = torch.Timer()

    av_L = 0

    -- mini batch loop
    for t = 1, 1 do
        -- Mini batch data
        inputs_batch = train_matrix:narrow(2,(t-1)*l+1,l)
        targets_batch = out_matrix:narrow(2,(t-1)*l+1,l)
        
        table = torch.split(targets_batch,1,2)
        target_table = {}
        for k, v in pairs (t) do
          target_table[k] = v:squeeze()
        end 
        
        -- reset gradients
        grad_params:zero()

        -- Forward pass (selection of inputs_batch in case the batch is not full, ie last batch)
        outputs = rrn:forward(inputs_batch:t()) 

        -- Average loss computation
        f = crit:forward(outputs, target_table)
        av_L = av_L +f

        -- Backward pass
        df_do = crit:backward(outputs,target_table)
        rnn:backward(inputs_batch, df_do)
        
        grad_params:view(4,1):renorm(1,2,5)
        
        rnn:updateParameters(eta)

    end

    print('Epoch '..i..': '..timer:time().real)
    print('Average Loss: '..av_L/math.floor(train_input:size(1)/b))

end

